In [2]:
import pandas as pd
import csv
import json
from pathlib import Path
import numpy as np
import math
from opencc import OpenCC
from datetime import datetime
import time
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import plotly.graph_objects as go

In [3]:
root_path = #deleted for security reasons
out_path = #deleted for security reasons
pic_path = #deleted for security reasons

# import and merge data

In [50]:
label_data = []
with open(out_path/'_cofacts_clustering_20220322-20220422.json' , 'r', encoding='big5') as reader:
    data = json.loads(reader.read())
    for idx, reply in enumerate(data):
        label_data.append([
            data[idx]['label'],
            data[idx]['size'],
            data[idx]['article_id'],
            data[idx]['covid'],
            data[idx]['coexist'],
            data[idx]['text'],
        ])

In [53]:
columns = ['label', 'size', 'article_id', 'covid', 'coexist', 'text']
label_data_all = pd.DataFrame(label_data, columns=columns)

In [54]:
article_truth_score_merge = pd.read_csv(out_path/'cofacts_truth_score_merge_20220322-20220422.csv')

In [64]:
cofacts_data = pd.read_excel(root_path/'cofacts_20220322-20220422_20220503dump.xlsx')

/opt/homebrew/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [66]:
cofacts_data = cofacts_data.rename(columns={'id':'article_id'})

In [70]:
for rows in cofacts_data.itertuples():
    cofacts_data.loc[rows.Index, 'createdAt'] = cofacts_data.loc[rows.Index, 'createdAt'].split('T')[0]

In [72]:
label_data_all = label_data_all.merge(cofacts_data[['article_id', 'createdAt']])

In [73]:
label_data_all = article_truth_score_merge[['article_id', 'article_type_prop']]\
                 .merge(label_data_all, how='right')

In [74]:
label_data = label_data_all[label_data_all.label.isna()==False]

In [75]:
label_data['label'] = label_data['label'].astype(int).astype(str)

/var/folders/6v/7554hy7n5zg4dv7nymqqshmm0000gn/T/ipykernel_6741/4072323723.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_data['label'] = label_data['label'].astype(int).astype(str)


# indicator 1: 每群裡面有幾種變體

In [197]:
label_data_covid = label_data[label_data.covid=='yes']

In [200]:
label_data_covid_size = label_data_covid.groupby(by='label', as_index=False).size()\
                       .rename(columns={'size':'label_size'}).sort_values(by='label_size', ascending=False)

In [211]:
label_data_covid_size.head(10)

,label,label_size
2803,583,14
2490,491,11
3815,872,7
1916,2313,7
2511,497,7
2531,503,7
4192,990,7
1002,1307,6
2497,493,6
3186,675,6


In [212]:
#回報則數
top10_post = label_data_covid_size.merge(label_data_covid, how='left')

In [230]:
top10_post.to_csv(out_path/'top10_covid_report_by_post.csv', index=False)

In [217]:
top10_post[top10_post.label=='503'].sort_values(by='createdAt')

,label,label_size,article_id,article_type_prop,size,covid,coexist,text,createdAt
52,503,7,31nmrg4rfiyed,謠言或個人意見,7.0,yes,no,我的媽啊，莫德納疫苗發明人竟然這樣說！恐怖！,2022-02-06
49,503,7,fis8isrzvlbn,謠言或個人意見,7.0,yes,no,莫德納疫苗發明人，竟然發表如此聲明。,2022-02-07
50,503,7,1bx3x1nr0r6ba,謠言或個人意見,7.0,yes,no,莫德納疫苗發明人這樣說！！,2022-02-07
51,503,7,2vbcg9rmpta8k,謠言或個人意見,7.0,yes,no,我的媽啊！“莫德納”疫苗發明人這樣說！做“父母”的如何選擇！這樣誰敢再打疫苗被當成試驗品啊！……,2022-02-07
48,503,7,vue2xoi0pzji,謠言或個人意見,7.0,yes,no,我的媽啊，莫德納疫苗發明人這樣說！！\n\n😰很重要一定要看🤭,2022-02-08
47,503,7,2ghuqwz591q5x,謠言或個人意見,7.0,yes,no,我的媽啊，莫德納疫苗發明人這樣說！\n尤其是小孩子，沒事最好不要接踵。,2022-02-09
46,503,7,2s6jxecri2mss,NaN,7.0,yes,no,莫德納疫苗發明人這樣說！！ 請你深思 考慮 您的孩子 要打疫苗嗎？??,2022-02-10


# 每篇文章在特定區間內有多少查詢瀏覽量

In [218]:
cofacts_total_views = pd.DataFrame(cofacts_data[['article_id', '2022-03-22', '2022-03-23',
                       '2022-03-24', '2022-03-25', '2022-03-26', '2022-03-27', '2022-03-28',
                       '2022-03-29', '2022-03-30', '2022-03-31', '2022-04-01', '2022-04-02',
                       '2022-04-03', '2022-04-04', '2022-04-05', '2022-04-06', '2022-04-07',
                       '2022-04-08', '2022-04-09', '2022-04-10', '2022-04-11', '2022-04-12',
                       '2022-04-13', '2022-04-14', '2022-04-15', '2022-04-16', '2022-04-17',
                       '2022-04-18', '2022-04-19', '2022-04-20', '2022-04-21', '2022-04-22']]\
                    .groupby(by='article_id').sum().T.sum()).reset_index().rename(columns={0:'view_size'})\
                    .sort_values(by='view_size', ascending=False)

In [219]:
covid_total_views = cofacts_total_views.merge(label_data_covid, how='right')\
                    .sort_values(by='view_size', ascending=False)

In [220]:
top10_covid_total_views = covid_total_views.head(10)

In [231]:
top10_covid_total_views.to_csv(out_path/'top10_covid_report_by_post_view.csv', index=False)

In [222]:
top10_covid_total_views

,article_id,view_size,article_type_prop,label,size,covid,coexist,text,createdAt
334,1earjxznpxpt7,11903,謠言或個人意見,701,2.0,yes,no,因應5/1勞動節及受疫情所影響之行業\n勞動部體恤全國勞工朋友 特編列2億特別預算 發放勞工...,2022-04-10
927,3w3ogi9ma0hin,4999,謠言或個人意見,2546,1.0,yes,no,在這個影片我已經完整呈現我的資歷，在疫苗的議題。我想我不必場場演講都說了\n\n呂丹宜博士/...,2022-01-29
508,3ct2h1dd84ush,4169,謠言或個人意見,1009,3.0,yes,no,（本文歡迎分享給更多人瞭解免疫學知識🙏）\n12月底時，我去益品書屋聽了蔡凱宙醫生關於人體免...,2022-01-22
504,2jpew6vb6m2a1,3056,謠言或個人意見,990,7.0,yes,no,真的假的？\n111年4月1號起。\n由於疫情的關係，勞動部決議，每人補助疫情援助金，新台幣...,2022-04-02
96,lvhsam7vg8qd,2670,謠言或個人意見,444,5.0,yes,no,感謝您分享關於「現在所謂的疫苗，根本不是疫苗。而只是一個實驗性的生物製劑。\n\n\n恭喜呂...,2022-01-07
80,20reqlsp93pyk,2413,謠言或個人意見,431,3.0,yes,no,呂丹宜教授的發言\n（前副總統呂秀蓮姪女）\n\n連呂副總統的侄女都戴好鋼盔了。\n\n恭喜...,2021-12-24
1259,3651v898tp9za,2216,NaN,4199,1.0,yes,no,她是勇敢正言的科學家。\n呂丹宜（前副總統呂秀蓮的姪女）獲得美國自然醫學醫師執照，並道破了「...,2022-02-06
1789,1if52ibqqbhu0,2152,NaN,6027,1.0,yes,no,轉自陳俊旭自然醫學博士臉書：\n伊維菌素真的很有效，但政府不准賣！\n\n我過去一年在美國診...,2022-01-23
464,33r3o3h5lwc85,2002,謠言或個人意見,928,5.0,yes,no,台大醫院護理長告訴大家如何防疫，保護自己！請大家耐心聽完？,2022-04-09
375,3jf2uivdeiaj9,1898,謠言或個人意見,786,3.0,yes,no,4／15 ～4／17 台灣全國三天吃素，祈求台灣疫情降下來，祈求大家平安，請大家一起來共...,2022-04-11


# 每群文章在特定區間內有多少查詢瀏覽量

In [223]:
covid_label_data_total_views = label_data_covid.merge(covid_total_views, how='left')

In [224]:
covid_label_data_total_views_sort = covid_label_data_total_views[['label', 'view_size']]\
                                    .groupby(by='label', as_index=False).sum()\
                                    .sort_values(by='view_size', ascending=False)\
                                    .rename(columns={'view_size':'label_view_size'})

In [238]:
covid_label_data_total_views_all = covid_label_data_total_views_sort.merge(covid_label_data_total_views)\
                                   .sort_values(by=['label_view_size', 'label'], ascending=False)

In [240]:
covid_label_data_total_views_all.to_csv(out_path/'covid_label_data_total_views_all.csv', index=False)

In [225]:
top_10_covid_label_data_total_views_sort = covid_label_data_total_views_sort.head(10)

In [226]:
top_10_covid_label_data_total_views_sort = top_10_covid_label_data_total_views_sort\
                                            .merge(covid_label_data_total_views)\
                                            .sort_values(by=['label_view_size', 'label'], ascending=False)

In [227]:
top_10_covid_label_data_total_views_sort.to_csv(out_path/'top10_covid_label_data_total_views_sort.csv', index=False)

In [235]:
top_10_covid_label_data_total_views_sort

,label,label_view_size,article_id,article_type_prop,size,covid,coexist,text,createdAt,view_size
0,701,13348,2wwc1iwqm5er8,謠言或個人意見,2.0,yes,no,因應5/1勞動節及受疫情所影響之行業\n司法部體恤全國司法院同仁 特編列2億特別預算 發放補...,2022-04-12,1445
1,701,13348,1earjxznpxpt7,謠言或個人意見,2.0,yes,no,因應5/1勞動節及受疫情所影響之行業\n勞動部體恤全國勞工朋友 特編列2億特別預算 發放勞工...,2022-04-10,11903
2,2546,4999,3w3ogi9ma0hin,謠言或個人意見,1.0,yes,no,在這個影片我已經完整呈現我的資歷，在疫苗的議題。我想我不必場場演講都說了\n\n呂丹宜博士/...,2022-01-29,4999
3,990,4674,2fij9jggngakx,謠言或個人意見,7.0,yes,no,110年5月20號起～\n由於疫情的關係，行政院決議，每人補助疫情援助金，新台幣28000元...,2021-05-21,2
4,990,4674,3l8pavcp7fcmr,謠言或個人意見,7.0,yes,no,"110年2月1號起。\n由於疫情的關係,行政院決\n議,每人補助疫情援助金,\n新台幣10,...",2021-05-08,12
5,990,4674,2bef8uyhqfyr1,謠言或個人意見,7.0,yes,no,好消息\n由於疫情的關係，行政院決議111年3月1號起開始申請，每人補助疫情援助金，新台幣1...,2022-02-18,404
6,990,4674,2brs1juiflssf,謠言或個人意見,7.0,yes,no,"111年2月10號起。\n由於疫情的關係，行政院決議，每人補助疫情援助金，新台幣10,000...",2022-02-10,1090
7,990,4674,30mhpoxnff9sq,謠言或個人意見,7.0,yes,no,111年4月1號起。\n由於疫情的關係，勞動部決議，每人補助疫情援助金，新台幣1萬元，申請詳...,2022-04-19,4
8,990,4674,1ci4aook2aloe,謠言或個人意見,7.0,yes,no,行政院 最新發布 111年4月30號底\n由於疫情的關係，行政院決議，每人補助疫情援助金，新...,2022-04-14,106
9,990,4674,2jpew6vb6m2a1,謠言或個人意見,7.0,yes,no,真的假的？\n111年4月1號起。\n由於疫情的關係，勞動部決議，每人補助疫情援助金，新台幣...,2022-04-02,3056
